In [15]:
import pylibxc
import numpy as np
import pandas as pd
from pathlib import Path

file_path = Path('negativity/gga_c.csv')
df = pd.read_csv(file_path, na_values='---')

file = file_path.stem
xc_col = file.replace('_c','').upper() + ' corr.'
df.rename(columns={'xc':xc_col}, inplace=True)


In [16]:
latex_parse = ['@article{', '@article {', '@inproceedings{', '@Inbook{']

bib_file = []
ref_names = []
for xc in df[xc_col]:
  func_c = pylibxc.LibXCFunctional(xc, "polarized")

  ref_name='\cite{'
  for ref in func_c.get_bibtex():
    bib_file.append(ref)
    latex_cite = ref.splitlines()[0]
    for repl in latex_parse:
      latex_cite = latex_cite.replace(repl,'')
    ref_name += latex_cite
    
  ref_name = ref_name[:-1] + '}' 

  ref_names.append(ref_name)

df['Refs.'] = ref_names 

# write out bib file
bib_file = "\n".join(bib_file)
out_file = Path(file_path.parent, file_path.name.replace('.csv','.bib'))
f = open(out_file, "w")
f.write(bib_file)
f.close

<function TextIOWrapper.close()>

In [17]:
# format functional names
df[xc_col] = df[xc_col].apply(lambda row: row.replace(file+'_','').replace('_','\_'))

# format columns
for c in df.columns:
  if 'range' in c:
    df[c] = df[c].apply(lambda row: np.NaN if row is np.NaN else np.array(row[1:-1].split(', '), dtype=np.float64))
    df[c] = df[c].apply(lambda row: np.around(row, decimals=4))

df = df.fillna('---')

In [18]:
with pd.option_context("max_colwidth", 1000):
    latex = df.to_latex(index=False, escape=False, column_format=(len(df.columns)-1)*'|c'+ '|l|')

# horizontal line sep
latex = latex.replace("\\\n", "\\ \hline\n").replace('\\toprule','\\hline')

latex_list = latex.splitlines()
latex=[]
for line in latex_list:
  if '\\toprule' in line or '\\midrule' in line or '\\bottomrule' in line:
    continue
  else:
    latex.append(line)



latex = "\n".join(latex)

out_file = Path(file_path.parent, file_path.name.replace('.csv','.tex'))
f = open(out_file, "w")
f.write(latex)
f.close

print(latex)

\begin{tabular}{|c|c|c|c|c|l|}
\hline
     GGA corr. &  satisfied condition? &      $r_s$ range &        $s$ range & $\zeta$ range  &                                                          Refs. \\ \hline
         acgga &                  True &              --- &              --- &            --- &                        \cite{Cancio2018_084116,Burke2014_4834} \\ \hline
        acggap &                  True &              --- &              --- &            --- &                        \cite{Cancio2018_084116,Burke2014_4834} \\ \hline
          am05 &                  True &              --- &              --- &            --- &                 \cite{Armiento2005_085108,Mattsson2008_084714} \\ \hline
          apbe &                  True &              --- &              --- &            --- &                                   \cite{Constantin2011_186406} \\ \hline
           bmk &                 False &    [0.0001, 5.0] &       [0.0, 5.0] &     [0.0, 1.0] &                      

In [19]:
# save updated 
out_file = Path(file_path.parent, file_path.name.replace('.csv','_table.csv'))
df.to_csv(out_file, index=None)